# Assessment of vancomycin vs. another antibiotic

In clinical practice, the pertinent question is often not "should I use vancomycin?", but rather "should I use vancomycin or this other antibiotic?". This notebook aims to quantify the risk of nephrotoxicity when using vancomycin over the other antibiotic.


## Definitions

* **drug on admission:** patient received medication order -12 to 12 hours upon admission to the ICU
* **baseline creatinine:** first creatinine value between -12 to 12 hours upon admission to the ICU
* **AKI:** following KDIGO guidelines using only creatinine, any instance of AKI between 2-7 days after their ICU admission.

KDIGO guidelines for AKI are: >= 50% change from baseline over 7 days, or absolute increase of 0.3 in creatinine over 48 hours.

## 0. Setup

In [ ]:
# Must install pandas-gbq. Link: https://pandas-gbq.readthedocs.io/en/latest/install.html#pip
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# helper functions stored in local py file
import utils

project_id='lcp-internal'

# Helper function to read data from BigQuery into pandas dataframes.
def run_query(query):
    return pd.io.gbq.read_gbq(query,
                              project_id=project_id, verbose=False,
                              dialect='standard')

# 1. Extract data and apply exclusions

For more detail on exclusions, see the main vancomycin analysis notebook.

In [ ]:
# cohort with exclusions applied
query = """
SELECT co.*
FROM `lcp-internal.vanco.cohort` co
"""
co = run_query(query)

# covariates from APACHE table
query = """
SELECT dem.*
FROM `hst-953-2018.team_i.demographics` dem
"""
dem = run_query(query)

# abx drug doses
query = "SELECT * FROM `lcp-internal.vanco.vanco`"
va = run_query(query)

query = "SELECT * FROM `lcp-internal.vanco.cefepime`"
ce = run_query(query)

query = "SELECT * FROM `lcp-internal.vanco.zosyn`"
zo = run_query(query)

# other drugs
query = "SELECT * FROM `lcp-internal.vanco.nsaids`"
nsaid = run_query(query)

query = "SELECT * FROM `lcp-internal.vanco.loop_diuretics`"
ld = run_query(query)

# AKI
query = """
SELECT 
  patientunitstayid
  , chartoffset
  , creatinine, creatinine_reference, creatinine_baseline
  , aki_48h, aki_7d
FROM `lcp-internal.vanco.aki`
"""
aki = run_query(query)

## Merge data

The antibiotic administration data extracted above has all administrations from time 0.
Thus, three dataframes need to be extracted before we can merge data:

1. Extract vanco_adm and vanco_wk
2. Extract cefepime_adm and cefepime_wk
3. Extract zosyn_adm and zosyn_wk

In [ ]:
v_df = utils.extract_adm_and_wk(va, 'vanco')
c_df = utils.extract_adm_and_wk(ce, 'cefepime')
z_df = utils.extract_adm_and_wk(zo, 'zosyn')

v_df.head()

Repeat the above for NSAIDs/loop diuretics.

In [ ]:
nsaid_df = utils.extract_adm_and_wk(nsaid, 'nsaid')
ld_df = utils.extract_adm_and_wk(zo, 'diuretic')
ld_df.head()

## Apply exclusions and create final dataframe

In [ ]:
# drop exclusions
idxKeep = co['patientunitstayid'].notnull()
for c in co.columns:
    if c.startswith('exclude_'):
        idxKeep = idxKeep & (co[c]==0)

# combine data into single dataframe
df = co.loc[idxKeep, ['patientunitstayid']].merge(dem, how='inner', on='patientunitstayid')

# add abx administration
df = df.merge(v_df, how='left', on='patientunitstayid')
df = df.merge(c_df, how='left', on='patientunitstayid')
df = df.merge(z_df, how='left', on='patientunitstayid')

# if the patient unit stay ID was not present for above join, they did not receive the abx
abx_columns = list(v_df.columns) + list(c_df.columns) + list(z_df.columns)
for c in abx_columns:
    df[c].fillna(0, inplace=True)
    df[c] = df[c].astype(int)

# add nsaid/loop diuretics
df = df.merge(nsaid_df, how='left', on='patientunitstayid')
df = df.merge(ld_df, how='left', on='patientunitstayid')

# if the patient unit stay ID was not present for above join, they did not receive the drug
d_columns = list(nsaid_df.columns) + list(ld_df.columns)
for c in d_columns:
    df[c].fillna(0, inplace=True)
    df[c] = df[c].astype(int)
    
aki_grp = aki.groupby('patientunitstayid')[['creatinine', 'aki_48h', 'aki_7d']].max()
aki_grp.reset_index(inplace=True)
df = df.merge(aki_grp, how='inner', on='patientunitstayid')

df['aki'] = ((df['aki_48h'] == 1) | (df['aki_7d'] == 1)).astype(int)

print('{} patients.'.format(df.shape[0]))

print('Antibiotic use on admission to ICU:')
for abx in ['vanco', 'cefepime', 'zosyn']:
    N = df[abx + '_adm'].sum()
    mu = N/df.shape[0]*100.0
    print(f'  {N:5d} ({mu:4.1f}%) - {abx}')
    
    
print('\nConcurrent antibiotic use:')
for abx1, abx2 in [['vanco', 'cefepime'], ['vanco', 'zosyn']]:
    N = ((df[abx1 + '_adm'] == 1) & (df[abx2 + '_adm'] == 1)).sum()
    mu = N/df.shape[0]*100.0
    print(f'  {N:5d} ({mu:4.1f}%) - {abx1} & {abx2}')
    

print('\nOther drug use on admission to ICU:')
for c in ['nsaid', 'diuretic']:
    N = df[c + '_adm'].sum()
    mu = N/df.shape[0]*100.0
    print(f'  {N:5d} ({mu:4.1f}%) - {c}')
    
df.head()

# Propensity matching

Define the dataframes used for (1) vanco only, (2) vanco + zosyn, and (3) vanco + cefepime.

### Vanco + Zosyn vs. Vanco + Cefepime

First, perform the comparison looking only at ICU admission drug administration.

In [ ]:
vanco_zosyn = df[(df['vanco_adm'] == 1) & (df['zosyn_adm'] == 1) & (df['cefepime_adm'] == 0)]
vanco_cefepime = df[(df['vanco_adm'] == 1) & (df['zosyn_adm'] == 0) & (df['cefepime_adm'] == 1)]
utils.match_and_print_or(exposure=vanco_zosyn, control=vanco_cefepime, seed=842)

Next, perform the same analysis requiring the administration to last at least from ICU admission to 48 hours post ICU admission.

In [ ]:
idx0 = (df['vanco_adm'] == 1) & (df['zosyn_adm'] == 1) & (df['cefepime_adm'] == 0)
idx1 = (df['vanco_wk'] == 1) & (df['zosyn_wk'] == 1) & (df['cefepime_wk'] == 0)
vanco_zosyn = df.loc[idx0 & idx1, :]

idx0 = (df['vanco_adm'] == 1) & (df['zosyn_adm'] == 0) & (df['cefepime_adm'] == 1)
idx1 = (df['vanco_wk'] == 1) & (df['zosyn_wk'] == 0) & (df['cefepime_wk'] == 1)
vanco_cefepime = df.loc[idx0 & idx1, :]

utils.match_and_print_or(exposure=vanco_zosyn, control=vanco_cefepime, seed=34246)

### Vanco vs. Vanco + Zosyn Analysis

In [ ]:
vanco_only = df[(df['vanco_adm'] == 1) & (df['zosyn_adm'] == 0)]
vanco_zosyn = df[(df['vanco_adm'] == 1) & (df['zosyn_adm'] == 1)]
utils.match_and_print_or(exposure=vanco_zosyn, control=vanco_only, seed=5513)

### Vanco vs. Vanco + Cefepime Analysis

In [ ]:
# Vanco + Cefepime Analysis
vanco_only = df[(df['vanco_adm'] == 1) & (df['cefepime_adm'] == 0)]
vanco_cefepime = df[(df['vanco_adm'] == 1) & (df['cefepime_adm'] == 1)]
utils.match_and_print_or(exposure=vanco_cefepime, control=vanco_only, seed=543289)

# Other drugs

We compare the use of vancomycin with the use of other drugs (either NSAIDs or loop diuretics).

## Unmatched comparisons

### Vanco vs. Vanco + NSAID (on admission)

In [ ]:
vanco_only = df[(df['vanco_adm'] == 1) & (df['nsaid_adm'] == 0)]
vanco_drug = df[(df['vanco_adm'] == 1) & (df['nsaid_adm'] == 1)]

print('\n=== Odds ratio of exposure ===\n')
diseased_exposed = len(vanco_drug[vanco_drug['aki'] == 1])
healthy_exposed = len(vanco_drug[vanco_drug['aki'] == 0])
diseased_nonexposed = len(vanco_only[vanco_only['aki'] == 1])
healthy_nonexposed = len(vanco_only[vanco_only['aki'] == 0])

utils.get_odds_ratio(diseased_exposed, healthy_exposed, diseased_nonexposed, healthy_nonexposed)

### Vanco vs. Vanco + NSAID (admission + week)

In [ ]:
idx0 = (df['vanco_adm'] == 1) & (df['nsaid_adm'] == 0)
idx1 = (df['vanco_wk'] == 1) & (df['nsaid_wk'] == 0)
vanco_only = df.loc[idx0 & idx1, :]

idx0 = (df['vanco_adm'] == 1) & (df['nsaid_adm'] == 1)
idx1 = (df['vanco_wk'] == 1) & (df['nsaid_wk'] == 1)
vanco_drug = df.loc[idx0 & idx1, :]

print('\n=== Odds ratio of exposure ===\n')
diseased_exposed = len(vanco_drug[vanco_drug['aki'] == 1])
healthy_exposed = len(vanco_drug[vanco_drug['aki'] == 0])
diseased_nonexposed = len(vanco_only[vanco_only['aki'] == 1])
healthy_nonexposed = len(vanco_only[vanco_only['aki'] == 0])

utils.get_odds_ratio(diseased_exposed, healthy_exposed, diseased_nonexposed, healthy_nonexposed)

### Vanco vs. Vanco + Loop diuretic (on admission)

In [ ]:
vanco_only = df[(df['vanco_adm'] == 1) & (df['diuretic_adm'] == 0)]
vanco_drug = df[(df['vanco_adm'] == 1) & (df['diuretic_adm'] == 1)]

print('\n=== Odds ratio of exposure ===\n')
diseased_exposed = len(vanco_drug[vanco_drug['aki'] == 1])
healthy_exposed = len(vanco_drug[vanco_drug['aki'] == 0])
diseased_nonexposed = len(vanco_only[vanco_only['aki'] == 1])
healthy_nonexposed = len(vanco_only[vanco_only['aki'] == 0])

utils.get_odds_ratio(diseased_exposed, healthy_exposed, diseased_nonexposed, healthy_nonexposed)

In [ ]:
idx0 = (df['vanco_adm'] == 1) & (df['diuretic_adm'] == 0)
idx1 = (df['vanco_wk'] == 1) & (df['diuretic_wk'] == 0)
vanco_only = df.loc[idx0 & idx1, :]

idx0 = (df['vanco_adm'] == 1) & (df['diuretic_adm'] == 1)
idx1 = (df['vanco_wk'] == 1) & (df['diuretic_wk'] == 1)
vanco_drug = df.loc[idx0 & idx1, :]

print('\n=== Odds ratio of exposure ===\n')
diseased_exposed = len(vanco_drug[vanco_drug['aki'] == 1])
healthy_exposed = len(vanco_drug[vanco_drug['aki'] == 0])
diseased_nonexposed = len(vanco_only[vanco_only['aki'] == 1])
healthy_nonexposed = len(vanco_only[vanco_only['aki'] == 0])

utils.get_odds_ratio(diseased_exposed, healthy_exposed, diseased_nonexposed, healthy_nonexposed)

## Matching on APACHE diagnosis

Only looks at ICU admission.

### Vanco vs. Vanco + NSAID (admission)

In [ ]:
vanco_only = df[(df['vanco_adm'] == 1) & (df['nsaid_adm'] == 0)]
vanco_zosyn = df[(df['vanco_adm'] == 1) & (df['nsaid_adm'] == 1)]
utils.match_and_print_or(exposure=vanco_zosyn, control=vanco_only, seed=64324)

### Vanco vs. Vanco + Loop diuretic (admission)

In [ ]:
vanco_only = df[(df['vanco_adm'] == 1) & (df['diuretic_adm'] == 0)]
vanco_zosyn = df[(df['vanco_adm'] == 1) & (df['diuretic_adm'] == 1)]
utils.match_and_print_or(exposure=vanco_zosyn, control=vanco_only, seed=4557)